# Prediction Insights

## Overview

Howso Engine enables powerful predictions with complete attribution and detailed explanations to make learning from
and debugging your data and predictions as easy as possible. For more information on predictions with Howso Engine,
check out the [predictions user guide](https://docs.howso.com/en/release-latest/user_guide/basic_capabilities/predictions.html).

In [1]:
from pprint import pprint

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Setup

This recipe will focus on the insights gained from predictions. Specifically, insights from:

- Influential Cases,
- Feature Contributions,
- Residual and Similarity Conviction,
- and Categorical Action Probabilities.

The [basic workflow guide](https://docs.howso.com/en/release-latest/user_guide/basic_capabilities/basic_workflow.html) covers creating a Trainee and performing a basic react in detail.

### Load Data and Create Trainee

In [2]:
df = pd.read_csv("../../data/iris/iris.tsv.gz", sep="\t", compression="gzip")
train_data = df.iloc[:-30]
new_data = df[~df.index.isin(train_data.index)]
features = infer_feature_attributes(train_data)

t = Trainee(features=features)

df

,Unnamed: 0,sepal length,sepal width,petal length,petal width,class
0,27,5.2,3.5,1.5,0.2,Iris-setosa
1,14,5.8,4.0,1.2,0.2,Iris-setosa
2,25,5.0,3.0,1.6,0.2,Iris-setosa
3,82,5.8,2.7,3.9,1.2,Iris-versicolor
4,97,6.2,2.9,4.3,1.3,Iris-versicolor
...,...,...,...,...,...,...
145,126,6.2,2.8,4.8,1.8,Iris-virginica
146,124,6.7,3.3,5.7,2.1,Iris-virginica
147,78,6.0,2.9,4.5,1.5,Iris-versicolor
148,125,7.2,3.2,6.0,1.8,Iris-virginica


### Train, Analyze, and React

In [3]:
t.train(train_data)
t.analyze()
t.react_into_features(similarity_conviction=True)

reaction = t.react(
    contexts=new_data,
    context_features=["sepal length", "sepal width", "petal length", "petal width"],
    action_features=["class"],
    details={
        "influential_cases": True,
        "similarity_conviction": True,
        "feature_robust_prediction_contributions": True,
        "feature_robust_residuals": True,
        "feature_full_residual_convictions_for_case": True,
        "categorical_action_probabilities": True,
    }
)

Note that, unlike in the basic workflow guide, we include several `details` in the react call and predict multiple cases as opposed to one. These are what will enable the
insights that we're going to get after the predictions are made.

For more information, see [the API documentation for `Trainee.react()`](https://docs.howso.com/en/release-latest/api_reference/_autosummary/howso.engine.html#howso.engine.Trainee.react).

### Inspect the Predictions

Howso Engine has high accuracy even on small datasets.

Here we see a comparison of the trained data (dots) and the predicted data (stars). This plot shows that the predictions align well with the trained data.

In [4]:
train_data = train_data.astype({"class": str})
predicted_data = pd.concat([new_data.reset_index(drop=True).drop(columns="class"), reaction["action"]], axis=1)
predicted_data = predicted_data.astype({"class": str})

cmap = px.colors.qualitative.D3
fig = go.Figure()
for i, (label, group) in enumerate(train_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        name=label,
        marker=dict(color=cmap[i], opacity=0.75),
        legendgroup="trained",
        legendgrouptitle_text="Trained class",
    ))

for i, (label, group) in enumerate(predicted_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        marker=dict(size=12, symbol="star", color=cmap[i], opacity=0.75),
        name=label,
        legendgroup="predicted",
        legendgrouptitle_text="Predicted class",
        hovertext=group.index,
    ))

fig.update_layout(
    xaxis_title="Petal Length",
    yaxis_title="Petal Width",
    title="Trained and Predicted Values"
)
fig.show(width=1000)

For categorical action features, the prediction can be further understood with the `categorical_action_probabilities` detail.  
This shows the probability of each possible value of the target feature. This information can highlight cases that are on the
border of two classes, like some of the above points are.  The closer a case gets to a class border, the more mixed the
categorical action probabilities may get.

In [5]:
pprint(reaction["details"]["categorical_action_probabilities"], compact=True)

[{'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-versicolor': 0.1615174054679077,
            'Iris-virginica': 0.8384825945320923}},
 {'class': {'Iris-versicolor': 0.05221136792962668,
            'Iris-virginica': 0.9477886320703732}},
 {'class': {'Iris-versicolor': 0.05613099247023369,
            'Iris-virginica': 0.9438690075297664}},
 {'class': {'Iris-versicolor': 0.0841344410237472,
            'Iris-virginica': 0.9158655589762528}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.09054803630690382,
            'Iris-virginica': 0.9094519636930961}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-versicolor': 0.060677974454419095,
            'Iris-virginica': 0.9393220255455809}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.1648712250435988,
            'Iris-virginica': 0.8351287749564011}},
 {'class': {'Iris-versicolor': 1}

For example, if we look at case `4`, we see a mix of probabilities because this case is on the border of two classes.

In [6]:
reaction["details"]["categorical_action_probabilities"][4]

{'class': {'Iris-versicolor': 0.0841344410237472,
  'Iris-virginica': 0.9158655589762528}}

### Insight 1: Which Cases Contributed?

Howso provides complete attribution for any and all predictions, showing exactly which cases influenced each prediction.  This can be used to understand or debug predictions.  For instance,
we can inspect the influential cases for one of the cases (case `4`) that was on the decision boundary in the plot above.  We can see that there is a mix of class values and no cases stand
out in terms of influence weight:

In [8]:
inf_cases_4 = reaction["details"]["influential_cases"][4]
inf_cases_4

,petal width,sepal length,class,sepal width,.influence_weight,.session,petal length,.session_training_index
0,1.8,6.3,Iris-virginica,2.9,0.231228,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,5.6,61
1,1.5,6.3,Iris-virginica,2.8,0.160353,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,5.1,99
2,1.8,6.4,Iris-virginica,3.1,0.116501,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,5.5,6
3,1.8,6.5,Iris-virginica,3.0,0.112520,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,5.5,66
4,1.5,6.0,Iris-virginica,2.2,0.089368,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,5.0,34
5,1.5,6.3,Iris-versicolor,2.5,0.084134,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,4.9,107
6,1.9,6.4,Iris-virginica,2.7,0.083284,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,5.3,103
7,2.1,6.4,Iris-virginica,2.8,0.072712,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,5.6,36
8,2.2,6.4,Iris-virginica,2.8,0.049899,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,5.6,46


Compare that to the influential cases for case `20`, which is firmly in the center of the solo cluster.  Here we see a single class value.

In [9]:
inf_cases_20 = reaction["details"]["influential_cases"][20]
inf_cases_20

,petal width,sepal length,class,sepal width,.influence_weight,.session,petal length,.session_training_index
0,0.2,5.2,Iris-setosa,3.5,0.081596,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,1.5,0
1,0.3,5.1,Iris-setosa,3.5,0.077784,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,1.4,48
2,0.2,5.2,Iris-setosa,3.4,0.076373,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,1.4,43
3,0.2,5.4,Iris-setosa,3.7,0.074842,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,1.5,104
4,0.4,5.0,Iris-setosa,3.4,0.073158,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,1.6,63
5,0.4,5.1,Iris-setosa,3.7,0.073110,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,1.5,117
6,0.2,5.4,Iris-setosa,3.4,0.070983,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,1.7,9
7,0.2,5.3,Iris-setosa,3.7,0.070674,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,1.5,68
8,0.2,5.1,Iris-setosa,3.4,0.070418,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,1.5,85
9,0.2,5.1,Iris-setosa,3.5,0.059560,62bbbef0-1f0c-4461-9807-a17d7d1e4fee,1.4,73


With the influential cases we can derive additional insights, such as identifying anomalous cases from within the influential cases using `similarity_conviction`.  
This can help to identify data that are making predictions more noisy or identify what type of data should be collected to improve predictive power in the future.

In [10]:
inf_case_indices = inf_cases_4[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"]
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
4,6.0,2.2,5.0,1.5,Iris-virginica,0.659944
1,6.3,2.8,5.1,1.5,Iris-virginica,0.886979
0,6.3,2.9,5.6,1.8,Iris-virginica,0.947569
5,6.3,2.5,4.9,1.5,Iris-versicolor,0.948574
7,6.4,2.8,5.6,2.1,Iris-virginica,1.089054
6,6.4,2.7,5.3,1.9,Iris-virginica,1.178328
2,6.4,3.1,5.5,1.8,Iris-virginica,1.217453
3,6.5,3.0,5.5,1.8,Iris-virginica,1.246210
8,6.4,2.8,5.6,2.2,Iris-virginica,1.302594


In [11]:
inf_case_indices = inf_cases_20[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"],
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
13,5.4,3.9,1.7,0.4,Iris-setosa,0.640131
7,5.3,3.7,1.5,0.2,Iris-setosa,0.806692
9,5.1,3.5,1.4,0.2,Iris-setosa,0.825878
3,5.4,3.7,1.5,0.2,Iris-setosa,0.858285
6,5.4,3.4,1.7,0.2,Iris-setosa,1.018131
14,5.4,3.9,1.3,0.4,Iris-setosa,1.101334
1,5.1,3.5,1.4,0.3,Iris-setosa,1.158660
10,5.1,3.3,1.7,0.5,Iris-setosa,1.178512
8,5.1,3.4,1.5,0.2,Iris-setosa,1.216573
11,5.0,3.4,1.5,0.2,Iris-setosa,1.353965


### Insight 2: Which Features Contributed?

In addition to providing attribution to cases, Howso also provides robust feature contributions to explain which features contributed to each prediction.
When inspecting the feature importances for case `13`,  we see that `petal width` and `petal length` provided the majority of the contribution to the 
prediction, whereas for case `20` `petal width` and `petal length` only have slightly higher contributions than the other features.  This could indicate 
that different features are more important for predicting different classes within this dataset and that focusing on those features would be prudent.

Identifying that certain cases that influenced a prediction are anomalous can help to explain low predictive power in certain parts of the data or other
issues that may appear during deployment.

In [14]:
fcs_13 = reaction["details"]["feature_robust_prediction_contributions"][13:14]
fcs_20 = reaction["details"]["feature_robust_prediction_contributions"][20:21]

display(fcs_13)
display(fcs_20)

,sepal length,petal width,sepal width,petal length
13,0.048279,0.289116,0.037363,0.28297


,sepal length,petal width,sepal width,petal length
20,0.072071,0.131137,0.049659,0.132093


### Insight 3: How Certain is the Prediction?

[Residuals](https://docs.howso.com/en/release-latest/user_guide/basic_capabilities/residuals.html) can characterize the uncertainty of the data around the prediction.  This will tell us which features are hard to predict in the region of the data around each case we're predicting.
If a prediction is less accurate than expected, this can explain which features were noisy and may have contributed to the problem.

In [15]:
reaction["details"]["feature_robust_prediction_contributions"]

,sepal length,petal width,sepal width,petal length
0,0.049922,0.279753,0.019713,0.366185
1,0.076697,0.282948,0.033695,0.199977
2,0.051224,0.284139,0.023982,0.234632
3,0.070142,0.314807,0.044967,0.238638
4,0.071869,0.260911,0.023013,0.308923
5,0.073659,0.239123,0.033809,0.233968
6,0.162201,0.200966,0.022321,0.196490
7,0.089213,0.216609,0.031949,0.210069
8,0.055829,0.240233,0.040917,0.339501
9,0.045666,0.176366,0.098590,0.174749


We can also use [`residual conviction`](https://docs.howso.com/en/release-latest/user_guide/basic_capabilities/conviction.html#prediction-residual-conviction) to determine which features
are uncertain in a scale-invariant manner, which can be useful if you wish to compare different features of different scales against each other.  Higher residual conviction
indicates something that is less surprising than expected and lower redsidual conviction indicates the opposite.

In [16]:
reaction["details"]["feature_full_residual_convictions_for_case"]

,petal width,sepal length,class,sepal width,petal length
0,1.620043,0.684588,1.000000,1.141282,0.661465
1,1.639118,1.026945,1.688377,0.534890,0.750473
2,0.545830,0.488922,0.358530,1.661608,0.468811
3,1.950145,0.988881,2.847446,0.662129,1.696923
4,0.306420,0.642931,2.388090,1.648205,0.618515
5,1.020112,1.399058,1.000000,0.601314,3.522708
6,1.972596,1.476192,1.000000,0.683898,0.965030
7,4.007417,0.782268,1.289495,0.617052,0.878756
8,0.814799,1.094119,8.235811,0.470829,2.220963
9,1.028817,1.514315,1.000000,0.418910,2.857726


### Insight 4: How Anomalous are the Predicted Cases?

By getting the [`similarity conviction`](https://docs.howso.com/en/release-latest/user_guide/basic_capabilities/conviction.html#similarity-conviction) of the cases that we predict, 
we can determine which of them are anomalous relative to the trained data. This could help to highlight cases that are unusually difficult or easy to predict, such as can be the case
when model drift occurs as a form of model monitoring.

In [17]:
pprint(reaction["details"]["similarity_conviction"], compact=True)

[1.5885883541728885, 1.0262407230392656, 1.0771631955065282, 0.9177208755569397,
 0.5096969011958339, 1.993100819133738, 1.7110597926523872, 1.0623247675296528,
 0.5847367099157665, 1.206863522878783, 2.080478369564731, 0.9139318318763775,
 3.2057931326111735, 1.1790362292645833, 0.9650101577162875, 0.99848044725525,
 0.7151767546225992, 1.36682789093241, 0.5451047581151172, 0.28829963204807413,
 0.6532725636836993, 1.826029773841103, 0.5581141260358722, 1.1512578500538029,
 1.2240128221504212, 1.3376656475718247, 1.229340729038162, 1.3964308642461913,
 0.8554964863397445, 0.9233197171898707]


In this case, there are no cases with a low similarity_conviction which indicates that there are no cases that we reacted to which are particularly
anomalous.